In [ ]:
import tensorflow as tf
import os 

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import cv2
from PIL import Image

In [ ]:
data_dir = 'data'

In [ ]:
image_exts = ['jpeg','jpg','bmg','png']

In [27]:
img = cv2.imread(os.path.join('data','happy','05-12-21-happy-people.jpg'))

In [28]:
img.shape

(880, 1400, 3)

In [30]:
for image_class in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, image_class)
    if not os.path.isdir(class_dir):
        continue
    for image in os.listdir(class_dir):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            if img is None:
                raise ValueError("Image could not be read")

            with Image.open(image_path) as img_pil:
                img_format = img_pil.format.lower()
            
            if img_format not in image_exts:
                print(f'Image not in ext list: {image_path}')
                os.remove(image_path)
        except Exception as e:
            print(f'Issue with image {image_path}: {e}')
            

Issue with image data\happy\05-12-21-happy-people.jpg: name 'Image' is not defined
Issue with image data\happy\1000_F_42220133_toAU6USGY9jVr2XJFLssfG00cSYIJ173.jpg: name 'Image' is not defined
Issue with image data\happy\11-Habits-of-Supremely-Happy-People-1.jpg: name 'Image' is not defined
Issue with image data\happy\170404-happy-workers-feature.jpg: name 'Image' is not defined
Issue with image data\happy\1920px-face-smile.svg_.png: name 'Image' is not defined
Issue with image data\happy\20150413185238-secrets-happy-entrepreneurs-woman-gratitude-rainbow-.jpeg: name 'Image' is not defined
Issue with image data\happy\25115.jpg: name 'Image' is not defined
Issue with image data\happy\35438_hd.jpg: name 'Image' is not defined
Issue with image data\happy\360_F_165246984_Ihe5LVattiq8zEPivcPqrtM85x7noWJw.jpg: name 'Image' is not defined
Issue with image data\happy\360_F_230964014_ngHsPlPbY2MCa0enKBex6YLNdTB24pki.jpg: name 'Image' is not defined
Issue with image data\happy\360_F_237330799_fyn

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
tf.keras.utils.image_dataset_from_directory??

In [44]:
data = tf.keras.utils.image_dataset_from_directory('data')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 28: invalid continuation byte